## Imports

In [ ]:
import logging
import queue
import random
import subprocess
import sys
import threading
import time
import weakref

import flask
import numpy as np
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import AjaxDataSource, ColumnDataSource, CustomJS
from bokeh.plotting import figure, show
from flask import Flask, jsonify, make_response, request
from flask.logging import default_handler
from ipywidgets import interact
from werkzeug.serving import make_server

from apache_beam.runners.interactive.display import data_server

## Parameters

In [ ]:
# Whether the notebook is running locally (including Colab) or on a remote server.
# If running on a remote server, all ports must be accessible from the internet.
LOCAL = True

In [ ]:
if LOCAL:
    HOST_IP = "localhost"
else:
    HOST_IP = subprocess.check_output(["hostname", "-I"], universal_newlines=True).strip().split()[0]

In [ ]:
logging.getLogger("werkzeug").setLevel(logging.WARNING)

## Dashboards

In [ ]:
def generate_plot(data_url):
    adapter = CustomJS(
        code="""
        const result = {x: [], y: []};
        const pts = cb_data.response;
        for (i=0; i<pts.length; i++) {
            result.x.push(pts[i][0])
            result.y.push(pts[i][1])
        }
        return result;
    """
    )

    source = AjaxDataSource(
        data_url=data_url, polling_interval=100, adapter=adapter, mode="append"
    )

    p = figure(
        plot_height=300,
        plot_width=800,
        background_fill_color="lightgrey",
        title="",
    )
    p.circle("x", "y", source=source)

    p.x_range.follow = "end"
    p.x_range.follow_interval = 10

    return p

### First sign wave

In [ ]:
data_queue = queue.Queue()

x = 0
y = np.sin(x) + np.random.random()

data_queue.put((x, y))

for i in range(10000):
    x += 0.1
    y = np.sin(x) + np.random.random()
    data_queue.put((x, y))

In [ ]:
endpoint = data_server.ServerThread(
    data_server.create_data_publisher_app(data_queue),
    host=("localhost" if LOCAL else "0.0.0.0"),
    port=0,
    threaded=False,
)

In [ ]:
endpoint.start()

In [ ]:
output_notebook(hide_banner=True)

data_url = "http://{}:{}/data".format(HOST_IP, endpoint.server.port)
show(generate_plot(data_url))

### Second sign wave

In [ ]:
data_queue2 = queue.Queue()

x = 0
y = np.sin(x) + np.random.random()

data_queue2.put((x, y))

for i in range(10000):
    x += 0.1
    y = np.sin(x) + np.random.random()
    data_queue2.put((x, y))

In [ ]:
endpoint2 = data_server.ServerThread(
    data_server.create_data_publisher_app(data_queue2),
    host=HOST_IP,
    port=0,
    threaded=False,
)

In [ ]:
endpoint2.start()

In [ ]:
output_notebook(hide_banner=True)

data_url = "http://{}:{}/data".format(HOST_IP, endpoint2.server.port)
show(generate_plot(data_url))

In [ ]:
from bokeh import plotting
doc = plotting.curdoc()
doc.clear()

doc = plotting.curdoc()
doc.clear()

In [ ]:
from bokeh import plotting

In [ ]:
plotting.reset_output()

In [ ]:
doc = plotting.curdoc()
doc.clear()